<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear Opamp 06   
# Inverting Mixer

<BR>
This project is a spin-off of the Inverting Amplifier project. We will see how to convert it to a signal mixer.

Version 1.2 (13/3/2019)  
License information is at the end of the document

---

**Bill Of Materials (BOM):**

* 1x Dual Opamp MCP6002
* Resistors: 2x $1 k\Omega$, $2,2k\Omega$ and 2x $33k\Omega$
* Capacitors: 2x $10\mu F$ (Electrolytic)

---

## The inverting mixer topology

The inverting mixer is based on the inverting amplifier. It is recommended to develop the work on this topology to understand this mixer. 

You can recall the inverting amplifier topology:

![Fig 01](images/L_OA_06/fig01.png)

As we remember, the **virtual shortcircuit** at the opamp inputs make $V_{(-)}$ always be at the ground zero voltage.
We can add one voltage source and one resistance to the circuit.

![Fig 02](images/L_OA_06/fig02.png)

In order to analyze this circuit we can see that, as $V_{(-)}$ voltage is zero, the current on resistors $R_2$ and $R_3$ is independent on each other. Also, in order to comply with the [Kirchhoff Current Law](https://en.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws), current at $R_1$ shall be the sum of currents at $R_2$ and $R_3$.

---

**CALCULATION TASK**  
Obtain the output voltage $V_O$ as function of $V_{i1}$, $V_{i2}$, $R1$, $R2$ and $R3$ for the above circuit using the virtual short circuit model.   

---
    
Observe that the output will include two contributions from the two inputs. Each contribution will have a negative gain that depends on a resistor ratio. If all resistors are equal:

$$R_1 = R_2 = R_3$$

The output voltage simplifies to:

$$V_o = -V_{i1}-V_{i2} = -(V_{i1}+V_{i2})$$

So we have mixed both input signals with the same weight. Using different R2 and R3 values we could mix the signals with different weights.

Remember that we cannot generate negative signals in the SLab system, so adding a reference voltage is in order:

![Fig 03](images/L_OA_06/fig03.png)

Remember also that it is recommended to have equal $R_5$ and $R_4$ values so that the reference voltage is at half the supply range.

We will choose the following resistor values:

$$R1 = 2.2 k\Omega \qquad R2 = R3 = 1 k\Omega \qquad R4 = R5 = 33 k\Omega$$

---

**CALCULATION TASK**  
Obtain the reference voltage $V_r$       
Obtain the output voltage $V_O$ as function of $V_{i1}$, $V_{i2}$ and $V_r$ for the proposed resistor values.  

---

## Measuring the circuit

Now we can measure the circuit. 

---

![Practical Icon](images/pt.png)

---

First, we import the **SLab** module and **connect** to the board.

In [ ]:
# Import modules
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board calibration.   
The following code cell checks the calibration of the **DACs** and the first four **ADCs**

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4)

Now we must build the circuit.

---

**BUILD TASK**  
Mount the proposed circuit  

---

The inputs are referred to the reference voltage $V_r$. So we will center our signals at $Vdd/2$. We can make SLab work out the values for us. Or we can also obtain the value from the circuit itself, as Vr is connected to **ADC2**

In [ ]:
vr = slab.readVoltage(2)
print('Vr','=',vr,'V')

Now we can load two different signals for the **DACs**. We will first generate a $0.2 V$ amplitude $100 Hz$ signal on **DAC1** using 200 points.

The wave on **DAC2** will be locked to the same sample frequency as the one in **DAC1**. In order to generate a signal on **DAC2** that has four times the frequency of the one in **DAC1**, we only need to provide $1/4$ of the data points. We will also select a $0.1 V$ amplitude for this signal.
The second parameter loads the wave in the wavetable associated to DAC.

In [ ]:
# Set a 100Hz sine wave on DAC1
slab.waveSine(vr-0.2,vr+0.2,200)
slab.setWaveFrequency(100)

# Set a 400Hz sine wave on DAC2
slab.waveSine(vr-0.1,vr+0.1,50,second=True)

We can now perform a reading of ADCs 1 to 3 for 2 waves (400 points) of the signal on **DAC1**. We use set the optional **dual** parameter of the waveResponse command to **True** so that not only the main wavetable is used on **DAC1** buy also the secondary wavetable is used on **DAC2**.

In [ ]:
# Store two primary waves for four ADC channels
slab.tranStore(400,4)

# Perform the measurement
t,wo,wr,w2,w1 = slab.waveResponse(dual=True)

# Plot the results
slab.plot1n(t,[w1,w2,wo],"Inverting Mixer"
             ,"time (s)","Signals (V)",["v1","v2","vo"])

Observe that the circuit performs the expected mixing of the signal. Observe also that there is a negative gain that is greater than one.

Mixing signals is tricky because we need enough dynamic range for all the signals we are mixing. If $v_1$ has $0.2 V$ amplitude, $v_2$ has $0.1 V$ amplitude and the amplifier gain is -2.2, you can work out the difference of the maximum and minimum voltages of the output signal.

You are not guaranteed to reach this limit as it depends on the alignment of peaks of the input signals. So, this is a upper bound of the output signal range.

You can obtain the current peak to peak of the output signal in order to compare its range with the calculated limit:

In [ ]:
# Obtain peak to peak output voltage
slab.peak2peak(vo)

Perform the requested measurements   
Check the peak to peak value of the output signal and compare it to the calculated maximum limit.    

If we wanted to add more inputs to the circuit we can add as many inputs as we want by adding new resistors connected to the new inputs and the $V_(-)$ node.

## DC Blocking

In the previous circuit, both inputs $V_{i1}$ and $V_{i2}$ were referred to the DC voltage on $V_r$. That enables us to mix both signals up to DC frequencies but need us to guarantee that the DC level of the signals is close enough to $V_r$ to prevent saturation of the $V_o$ output.

For small gains it is easy to have a DC level close enough to $V_r$ but it is more difficult at higher gains. 

The following circuit adds capacitors $C_2$ and $C_3$ in series with resistors $R_2$ and $R_3$. They block the DC level of the inputs make because a zero at zero frequency is added at both inputs.

![Fig 04](images/L_OA_06/fig04.png)

In the same way as in the Inverting Amplifier we will have a low frequency cutoff at the frequency defined by a pole. As we have two inputs $V_{i1}$ and $V_{i2}$, we have one pole for each input: $p_{L1}$ for $V_{i1}$ and $p_{L2}$ for $V_{i2}$. If both resistors and capacitors are equal, both poles will be also the same.

$$p_{L1} = \frac{1}{R_2 C_2}
\qquad p_{L1} = \frac{1}{R_3 C_3}
\qquad poles \; in \; \frac{rad}{s}$$

We propose to keep the $R_2$ and $R_3$ values and use $C_2 = C_3 = 10 \mu F$.

---

**CALCULATION TASK**  
Obtain the pole positions, in Hz, for the selected component values.     

---

Capacitors $C_2$ and $C_3$ are probably **electrolytic**, in that case, you should not reverse its bias voltage. That means using input voltages with DC value below $V_r$.

You can set DACs to zero before mounting the circuit to guarantee you don't reverse polarize the capacitors:

In [ ]:
# Set DACs to zero
slab.zero()

---

**BUILD TASK**  
Mount the proposed circuit  

---

We can now test the circuit using the same signals but preventing reverse polarize the caps.

In [ ]:
# Set primery and secondary waves
slab.waveSine(vr-0.6,vr-0.2,200)
slab.waveSine(vr-0.4,vr-0.2,50,second=True)

# Measure and plot
slab.wavePlot(dual=True)

The output shall be independent on the DC value of the input signals as we have the DC blocking capacitors. We can check that:

In [ ]:
# Change of input wave positions
slab.waveSine(1-0.2,1+0.2,200)
slab.waveSine(0.5-0.1,0.5+0.1,50,second=True)
slab.wavePlot(dual=True)

Remember that we have a low frequency zero, if we use a frequency too low, we will attenuate the signals.

In [ ]:
# Measurement for 10Hz primary wave
slab.setWaveFrequency(10)
slab.wavePlot(dual=True)

Attenuation will further increase if we lower more the frequency:

In [ ]:
# Measurement for 2Hz primary wave
slab.setWaveFrequency(2)
slab.wavePlot(dual=True)

Observe that, as signals generated by **DAC1** and **DAC2** have different frequencies, the attenuation is different for each signal contribution.

## Last comments

This was a short project. We have started from the **Inverting Amplifier** topology and we have built a **mixer** around it.  

It is quite usual to build new circuits on the concepts or previous more simple circuits.

## References

[SLab Python References](../Reference)  
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module. 
They should be available in the SLab/Doc folder.

[TinyCad](https://sourceforge.net/projects/tinycad/)  
Circuit images on this document have been drawn using the free software TinyCad  

[SciPy](https://www.scipy.org/)  
All the functions plots have been generated using the Matplotlib SciPy package.

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">